In [9]:
# Importar las librerías necesarias para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import warnings

# Ignorar advertencias (warnings) para mantener una salida limpia
warnings.filterwarnings("ignore")

# Cargar el dataframe limpio.
df_cleaned = pd.read_csv('datos/df_cleaned.csv')

# Haremos una copia para mantener el original intacto.
df_featured = df_cleaned.copy()